In [1]:
# This cell contains some utility functions to prepare and execute the benchmarks
import timeit
from statistics import median
from random import choice
from string import ascii_uppercase

In [3]:
def random_string(length):
    """Produce a random string made of *length* uppercase ascii characters"""
    return ''.join(choice(ascii_uppercase) for i in range(length))

In [20]:
def print_scaling(stmt, setup, sizes=[10000, 20000, 30000], repeat=False, units='us'):
    """Print scaling information for the statement *stmt*, executed after *setup*.
    
    The *setup* and *stmt* arguments take a template string where "{N}"
    will be replaced as the size of the input.
    
    The *repeat* flags determined if the setup needs to be run between
    each test run.
    """
    values = []
    for size in sizes:
        if repeat:
            timings = timeit.repeat(stmt.format(N=size),
                                    setup=setup.format(N=size),
                                    number=1, repeat=1000)
            values.append(min(timings))
        else:
            timings = timeit.repeat(stmt.format(N=size),
                                    setup=setup.format(N=size),
                                    number=1000, repeat=3)
            values.append(min(t/1000 for t in timings))
    unit_factor = {'us': 1e6,
                   'ms': 1e3}[units]
    
    print(stmt, '\t | ', ' | '.join('N = {} t = {:.2f} ({})'.format(n, t * unit_factor, units) for n, t in zip(sizes, values)))

# List operations - print time taken to perform operations on List

In [21]:
list_operations = [
    'collection.pop()',
    'collection.pop(0)', 
    'collection.append(1)',
    'collection.insert(0, 1)'
]

In [22]:
for operation in list_operations:
    print_scaling(operation,
                  setup='collection = list(range({N}))')

collection.pop() 	 |  N = 10000 t = 0.07 (us) | N = 20000 t = 0.06 (us) | N = 30000 t = 0.06 (us)
collection.pop(0) 	 |  N = 10000 t = 2.26 (us) | N = 20000 t = 5.29 (us) | N = 30000 t = 6.78 (us)
collection.append(1) 	 |  N = 10000 t = 0.04 (us) | N = 20000 t = 0.04 (us) | N = 30000 t = 0.04 (us)
collection.insert(0, 1) 	 |  N = 10000 t = 3.31 (us) | N = 20000 t = 6.55 (us) | N = 30000 t = 9.83 (us)


# Deque operations - Evaluate time taken to perform operation on deque 

In [30]:
deque_operations = [
    'collection.pop()', 
    'collection.append(1)', 
    'collection.popleft()', 
    'collection.appendleft(1)', 
    'collection[0]',
    'collection[{N}-1]',
    'collection[int({N} / 2)]'
]

In [31]:
for operation in deque_operations:
    print_scaling(operation, 
                  setup='from collections import deque; collection = deque(range({N}))')

collection.pop() 	 |  N = 10000 t = 0.08 (us) | N = 20000 t = 0.08 (us) | N = 30000 t = 0.07 (us)
collection.append(1) 	 |  N = 10000 t = 0.07 (us) | N = 20000 t = 0.06 (us) | N = 30000 t = 0.06 (us)
collection.popleft() 	 |  N = 10000 t = 0.05 (us) | N = 20000 t = 0.05 (us) | N = 30000 t = 0.05 (us)
collection.appendleft(1) 	 |  N = 10000 t = 0.05 (us) | N = 20000 t = 0.05 (us) | N = 30000 t = 0.05 (us)
collection[0] 	 |  N = 10000 t = 0.03 (us) | N = 20000 t = 0.03 (us) | N = 30000 t = 0.03 (us)
collection[{N}-1] 	 |  N = 10000 t = 0.03 (us) | N = 20000 t = 0.03 (us) | N = 30000 t = 0.03 (us)
collection[int({N} / 2)] 	 |  N = 10000 t = 0.17 (us) | N = 20000 t = 0.26 (us) | N = 30000 t = 0.33 (us)


# Bisect - place element in sorted array 

In [37]:
import bisect
my_ordered_list = ['a', 'b', 'c', 'd', 'f']
# my_ordered_list = [1,2,3,4,6]
print(f"element to be placed at", bisect.bisect(my_ordered_list, 'e'))


element to be placed at 4


In [38]:
bisect.insort(my_ordered_list, 'e')

In [39]:
my_ordered_list

['a', 'b', 'c', 'd', 'e', 'f']

In [42]:
def index_bisect(a, x): 
    'locate the leftmost value exactly equal to x'
    i = bisect.bisect_left(a, x)

    if i != len(a) and a[i] == x: 
        return i 
    raise ValueError


# compare list element retrieval vs bisect

In [43]:
setup_code = '''
import random
random.seed(42)

collection = list(range({N}))
'''
print_scaling('collection.index(random.randint(0, {N}))',
              setup=setup_code)

collection.index(random.randint(0, {N})) 	 |  N = 10000 t = 61.82 (us) | N = 20000 t = 122.91 (us) | N = 30000 t = 181.51 (us)


In [44]:
setup_code = '''
from __main__ import index_bisect

import random
random.seed(42)

collection = list(range({N}))
'''
    
print_scaling('index_bisect(collection, random.randint(0, {N}))',
              setup=setup_code)

index_bisect(collection, random.randint(0, {N})) 	 |  N = 10000 t = 1.07 (us) | N = 20000 t = 1.10 (us) | N = 30000 t = 1.09 (us)


# Dictionary

In [45]:
def counter_dict(items):
    """ count number of unique elements in list """

    counter = {} 

    for item in items: 
        if item not in counter:
            counter[item] = 1
        else: 
            counter[item] += 1
    return counter

In [59]:
# test the function 
import random 
random.seed(42)
# get list of numbers between 0 to 100 
data_list = [random.randint(0, 100) for i in range(1000)]
# print first 20 records
data_list[:20]

[81, 14, 3, 94, 35, 31, 28, 17, 94, 13, 86, 94, 69, 11, 75, 54, 4, 3, 11, 27]

In [67]:
counter_dict(data_list[:20])

{81: 1,
 14: 1,
 3: 2,
 94: 3,
 35: 1,
 31: 1,
 28: 1,
 17: 1,
 13: 1,
 86: 1,
 69: 1,
 11: 2,
 75: 1,
 54: 1,
 4: 1,
 27: 1}

In [49]:
from collections import defaultdict
def counter_defaultdict(items):
    # create a dictionary where each element is assigned to value zero 
    counter = defaultdict(int)

    for item in items:
        counter[item] += 1

    return counter

In [68]:
counter_defaultdict(data_list[:20])

defaultdict(int,
            {81: 1,
             14: 1,
             3: 2,
             94: 3,
             35: 1,
             31: 1,
             28: 1,
             17: 1,
             13: 1,
             86: 1,
             69: 1,
             11: 2,
             75: 1,
             54: 1,
             4: 1,
             27: 1})

In [69]:
from collections import Counter 
Counter(data_list[:20])

Counter({81: 1,
         14: 1,
         3: 2,
         94: 3,
         35: 1,
         31: 1,
         28: 1,
         17: 1,
         13: 1,
         86: 1,
         69: 1,
         11: 2,
         75: 1,
         54: 1,
         4: 1,
         27: 1})

In [50]:
setup_code = '''
from uuid import uuid4
from collections import Counter
from __main__ import counter_defaultdict, counter_dict
import random

random.seed(42)
collection = [random.randint(0, 100) for i in range({N})]
'''

In [53]:
print_scaling('counter_dict(collection)',
              setup=setup_code,
              sizes=[1000, 2000, 3000])
print_scaling('counter_defaultdict(collection)', 
              setup=setup_code,
              sizes=[1000, 2000, 3000])
print_scaling('Counter(collection)', 
              setup=setup_code, 
              sizes=[1000, 2000, 3000])

counter_dict(collection) 	 |  N = 1000 t = 72.52 (us) | N = 2000 t = 144.71 (us) | N = 3000 t = 217.25 (us)
counter_defaultdict(collection) 	 |  N = 1000 t = 62.07 (us) | N = 2000 t = 115.61 (us) | N = 3000 t = 167.12 (us)
Counter(collection) 	 |  N = 1000 t = 25.34 (us) | N = 2000 t = 48.71 (us) | N = 3000 t = 73.02 (us)


# inverted indexes - Find the matching word in the documents

In [70]:
docs = [
    "the cat is under the table", 
    "the dog is under the table", 
    "cat and dog smell roses", 
    "Carla eats apple"
]

In [71]:
def build_inverted_index(docs):
    index = {} 

    for i, doc in enumerate(docs): 
        for word in doc.split():
            if word not in index:
                index[word] = [i]
            else: 
                index[word].append(i)

    return index

In [72]:
doc_index = build_inverted_index(docs)

In [86]:
def find_the_word(word, doc_index=doc_index): 
    if word in doc_index:
        return doc_index[word]
    return []

In [87]:
def get_the_document(index_list, docs=docs):
    result = []
    if len(index_list) > 0:
        for index in index_list:
            if index < len(docs):
                result.append(docs[index])
        return result
    raise ValueError

In [89]:
find_the_word("cat")
get_the_document(find_the_word("table"))

['the cat is under the table', 'the dog is under the table']

# Inverted Index using set for multiple items in query

In [90]:
def build_inverted_index_using_set(docs):
    index = {} 

    for i, doc in enumerate(docs): 
        for word in doc.split():
            if word not in index:
                index[word] = {i}
            else: 
                index[word].add(i)
    return index

doc_index_set = build_inverted_index_using_set(docs)

{'the': {0, 1},
 'cat': {0, 2},
 'is': {0, 1},
 'under': {0, 1},
 'table': {0, 1},
 'dog': {1, 2},
 'and': {2},
 'smell': {2},
 'roses': {2},
 'Carla': {3},
 'eats': {3},
 'apple': {3}}

In [95]:
find_the_word("under", doc_index=doc_index_set)

{0, 1}

In [96]:
find_the_word("table", doc_index=doc_index_set)

{0, 1}

In [97]:
get_the_document(find_the_word("under", doc_index=doc_index_set).intersection(find_the_word("table", doc_index=doc_index_set)))

['the cat is under the table', 'the dog is under the table']

# Heaps - to identify maximum and minimum values

In [98]:
import heapq 

In [ ]:
collection = [10, 3, 4, 2, 3, 5, 6, 8, 1]
heapq.heapify(collection)

In [104]:
min_val = heapq.heappop(collection)
min_val

1

#### Get max value

In [109]:
def mark_element_negative(list_item):
    return [-1 * i for i in list_item]

In [113]:
# multiply each element by -1 and get the get the minium -> convert it back by multiplying -1 again 
collection = [10, 3, 4, 2, 3, 5, 6, 8, 1]
collection = mark_element_negative(collection)
heapq.heapify(collection)
heapq.heappop(collection) * -1

10

#### using priority queue to get minimum value

In [120]:
from queue import PriorityQueue

def add_elements_to_queue(list_data):
    min_queue = PriorityQueue()
    max_queue = PriorityQueue()
    for element in list_data:
        min_queue.put(element)
        max_queue.put(-1*element)
    
    return min_queue, max_queue

def get_min_value_queue(min_queue, max_queue, switch='max'):
    if switch=='max':
        return max_queue.get() * -1
    
    return min_queue.get()


collection = [10, 3, 4, 2, 3, 5, 6, 8, 1]
min_queue, max_queue = add_elements_to_queue(collection)
get_min_value_queue(min_queue, max_queue, switch='min')

1

In [127]:
# store the task associated along with data 
def rand_int_generator(): 
    random.seed(42)
    return random.randint(0, 100)

def hello_world():
    return 'hello world'

def random_string_generator():
    random.seed(42)
    return random_string(10)

queue = PriorityQueue()
queue.put((3, hello_world()))
queue.put((2, rand_int_generator()))
queue.put((1, random_string_generator()))

print(queue.get()[1])
print(queue.get()[1])
print(queue.get()[1])


UDAXIHHEXD
81
hello world


# Tries - 

In [128]:
from random import choice
from string import ascii_uppercase 

def random_string_trie(length):
    """ Create random string of given length in uppercase """
    return ''.join(choice(ascii_uppercase) for i in range(length))

string_list = [random_string_trie(32) for i in range(10000)]
string_list[:10]

['VXRCSNBACGHQTARGWUWRNHOSIZAYZFWN',
 'KIEGYKDCMDLLTIZBXORDMCRJUTLSGWCB',
 'VHYJCHDMIOULFLLGVIWVUCTUFRXHFOMI',
 'UWRHVKYYBHBZKMICGSWKGUPMUOEIEHXR',
 'RIXSNSMLHEQPCYBDEUFZVNTCMMTOQIRA',
 'VXDVRYIYUKDJNFOAXXIQYFQDUJUQTGEL',
 'YFRYQATKPADLZJHBHSCCXPCYRYEEVPRF',
 'IQTNGRYXWGWJMVULOQODHHCKASRHSHAC',
 'WUBHCBKCQHIVPGREXSSPHZPZNGDDVNLN',
 'NOXBVUUDBMXKZDHGGROENFIOHCOZRDBU']

In [130]:
# get the matching values 
matches = [s for s in string_list if s.startswith('AA')]
matches

['AACWQMONMDBAQUTVRZVPWMDARCJGLMZC',
 'AACRVVEAQPDMUGQRDSGWVSUUEPCSDQSS',
 'AAICIVGFUYWNXVUTJUJDZAIYEGEABWVR',
 'AAPIRQHQENREBCEBEWZDKZEMFBPVVYNV',
 'AAKGFLRYSPJRZMTPCFCANTXQGLOOHSOR',
 'AAPHRRTZKAUUDRAVZKCAYYLDWXCSGCYW',
 'AARSZAWWMNANLPVAQQZIDWGUZMDBLTUZ',
 'AADFTZUTESFUNGRQBCUOZGJSHQEOGQHP',
 'AAIUZGVHVKNIXOBXOCGWKJNHVCFVZEVD',
 'AAYUEQUUVTLBISFUFGHHRURTLLZEICUU']

In [131]:
# timeit the the search algo 
%timeit [s for s in string_list if s.startswith('AA')]

995 µs ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [132]:
from patricia import trie
string_dict = {s:0 for s in string_list}
string_trie = trie(**string_dict)
string_trie

trie({'VXRCSNBACGHQTARGWUWRNHOSIZAYZFWN': 0, 'VXRYFNUGEREYBOAMOFBVBAORDMZVDCVZ': 0, 'VXDVRYIYUKDJNFOAXXIQYFQDUJUQTGEL': 0, 'VXDHVIZMOLNHLDBIDODEBDJDYDMWGNLG': 0, 'VXMOYTSQCHNJZMEEKCPSXBRMICMQNYWE': 0, 'VXMPAZRIQYQMUQXYNZKMWVESFLYVVRVF': 0, 'VXOMVRHXDMBRFWODUPFCOVGGOZWXGSUS': 0, 'VXOUBMDTQPRXBUCROZVCNMUSSGIUCBKK': 0, 'VXEPJMTFZCDRVZUREICTWSMENIFFETPK': 0, 'VXLTFBKXZLSNOKOPTTCDVGZOSZRHAWJY': 0, 'VXLATURLITSCLPTXLMZCSKPFDYWYZPQY': 0, 'VXCJUQKKXZAOAEMIANEZZNAMOJWDKUDX': 0, 'VXKRDQDUCAMWTTJVEZZSFFHCXTPXQSWK': 0, 'VXQIATSWDFYMTCOMYISRLOOJFOGKSRIB': 0, 'VXJPRXSVCGNPDNCUNQJYSVQUXSTUCUEM': 0, 'VXGXQVMKBGBMUYMMUKUKZPKDRIWZEGQY': 0, 'VXYCKNFPRXGCZEHZIETVUTFMIPMUTNKB': 0, 'VXYLSFRPOCCBTKZLCVAZDZIXKHHKXYQQ': 0, 'VXIECOHUBYRANQNJBZHKNWVGBUPPGSEZ': 0, 'VXSIHAHTYCSNAVGBFJVOVFDJLAUWDIAM': 0, 'VHYJCHDMIOULFLLGVIWVUCTUFRXHFOMI': 0, 'VHYDZLDWACIUSHKBARRYTZFLPRKBMGDK': 0, 'VHCTDAATBAIAJOMXSFDBLGUGNXLCINUA': 0, 'VHWJAWFJEUFUFXIDWTQZXHZZBNVHFXHN': 0, 'VHLGZNPHLVZLWQTNBPBOEERRGIYCGHZL': 0, 'VHDNDHVJAXPSTEZVXC

In [135]:
matches = list(string_trie.iter('AA'))
matches

['AACWQMONMDBAQUTVRZVPWMDARCJGLMZC',
 'AACRVVEAQPDMUGQRDSGWVSUUEPCSDQSS',
 'AAICIVGFUYWNXVUTJUJDZAIYEGEABWVR',
 'AAIUZGVHVKNIXOBXOCGWKJNHVCFVZEVD',
 'AAPIRQHQENREBCEBEWZDKZEMFBPVVYNV',
 'AAPHRRTZKAUUDRAVZKCAYYLDWXCSGCYW',
 'AAKGFLRYSPJRZMTPCFCANTXQGLOOHSOR',
 'AARSZAWWMNANLPVAQQZIDWGUZMDBLTUZ',
 'AADFTZUTESFUNGRQBCUOZGJSHQEOGQHP',
 'AAYUEQUUVTLBISFUFGHHRURTLLZEICUU']

In [136]:
%timeit list(string_trie.iter('AA'))

7.89 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Caching

In [141]:
from functools import lru_cache

# set 16 size i.e. it can cache up to 16 results 
@lru_cache (maxsize=16)
def sum2(a,b):
    print(f"Calculating {a} + {b}")
    return a + b

print(sum2(2,1))

Calculating 2 + 1
3


In [142]:
print(sum2(3,4))

Calculating 3 + 4
7


Notice no print for calculating as result from the function was cached

In [143]:
print(sum2(2,1))

3


In [ ]:
# get cache information 
sum2.cache_info()

CacheInfo(hits=1, misses=2, maxsize=16, currsize=2)

### Febonacci series implementation w/o using caching

In [147]:
def febo(n):
    if n < 1:
        return 1
    return febo(n-1) + febo(n-2)

febo(20)

17711

In [168]:
%timeit febo(22)

7.1 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Febonacci series implementation using caching

In [167]:
import timeit

setup_code = """ 
from functools import lru_cache
from __main__ import febo 
febo_memoized = lru_cache(maxsize=None) (febo)
"""

# notice lru_cacle(max_size=None) febo is equivalent to decorator used as 
# @lru_cache
# def febo(n):
#   ... 

results = timeit.repeat('febo_memoized(22)', 
                        setup=setup_code, 
                        repeat=1000, 
                        number=1)
print("Fibonacci took max {:.2f} us and min {:.2f}".format(max(results), min(results)))



Fibonacci took max 0.04 us and min 0.01


In [169]:
@lru_cache(maxsize=None)
def febo(n):
    if n < 1:
        return 1
    return febo(n-1) + febo(n-2)

febo(22)

46368

In [170]:
%timeit febo(22)

53 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Joblib for caching

In [173]:
from joblib import Memory

memory = Memory(location='/Users/rohitabhishek/Documents/Programming_Workspace/Github-Programs/advanced-python/Chapter_2/data')

@memory.cache
def sum2(a, b):
    print(f"Calculating {a} + {b}")
    return a + b
    

In [174]:
sum2(1,5)

________________________________________________________________________________
[Memory] Calling __main__--var-folders-kx-s0nzgxq96zgdm61_51783vnm0000gn-T-ipykernel-4165690985.sum2...
sum2(1, 5)
Calculating 1 + 5
_____________________________________________________________sum2 - 0.0s, 0.0min


6

In [175]:
sum2(2,4)

________________________________________________________________________________
[Memory] Calling __main__--var-folders-kx-s0nzgxq96zgdm61_51783vnm0000gn-T-ipykernel-4165690985.sum2...
sum2(2, 4)
Calculating 2 + 4
_____________________________________________________________sum2 - 0.0s, 0.0min


6

In [176]:
sum2(1,5)

6

# Comprehensions and Generator

In [177]:
def loop(n=100000):
    res = [] 
    for i in range(n):
        res.append(i * i)

    return sum(res)

In [183]:
loop()

333328333350000

In [178]:
%timeit loop()

7.43 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [179]:
def comprehension(n=100000):
    return sum([i*i for i in range(n)])


In [184]:
comprehension()

333328333350000

In [180]:
%timeit comprehension()

5.41 ms ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [186]:
def generator(n=100000):
    yield sum(i*i for i in range(n))


In [192]:
# generator()
next(generator())

333328333350000

In [194]:
%timeit generator()

172 ns ± 0.114 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [230]:
def map_comprehension(numbers=range(1000000)):

    a = [n*2 for n in numbers]
    b = [n**2 for n in a]
    c = [n**0.33 for n in b]

    return max(c)

In [231]:
map_comprehension()

14410.515288219276

In [232]:
%timeit map_comprehension()

372 ms ± 2.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [233]:
def map_normal(numbers=range(1000000)):
    a = map(lambda n: n*2, numbers)
    b = map(lambda n: n**2, a)
    c = map(lambda n: n** 0.33, b)

    yield max(c)

In [234]:
next(map_normal())

14410.515288219276

In [236]:
%timeit map_normal()

174 ns ± 0.216 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [237]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [238]:
%memit map_comprehension()

peak memory: 328.98 MiB, increment: 0.00 MiB


In [239]:
%memit map_normal()

peak memory: 329.14 MiB, increment: 0.01 MiB
